# Mistralモデルでの開発

## はじめに

このレッスンでは以下の内容を扱います：
- さまざまなMistralモデルの紹介
- 各モデルのユースケースや利用シーンの理解
- 各モデルの特徴を示すコードサンプル


## Mistralモデルについて

このレッスンでは、3つの異なるMistralモデル、**Mistral Large**、**Mistral Small**、そして**Mistral Nemo**を紹介します。

これらのモデルはすべて、Github Modelマーケットプレイスで無料で利用できます。このノートブックのコードは、これらのモデルを使って実行されます。Github Modelsを使って[AIモデルでプロトタイピングする方法](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)について、さらに詳しく知りたい方はこちらをご覧ください。


## Mistral Large 2 (2407)
Mistral Large 2は、現在Mistralのフラッグシップモデルであり、エンタープライズ向けに設計されています。

このモデルは、オリジナルのMistral Largeから以下の点でアップグレードされています。
- より大きなコンテキストウィンドウ - 128k（従来は32k）
- 数学やコーディングタスクでのパフォーマンス向上 - 平均正答率76.9%（従来は60.4%）
- 多言語対応の強化 - 対応言語：英語、フランス語、ドイツ語、スペイン語、イタリア語、ポルトガル語、オランダ語、ロシア語、中国語、日本語、韓国語、アラビア語、ヒンディー語

これらの特徴により、Mistral Largeは以下の用途で優れた性能を発揮します。
- *Retrieval Augmented Generation (RAG)* - より大きなコンテキストウィンドウにより、RAGに最適
- *Function Calling* - このモデルはネイティブで関数呼び出しに対応しており、外部ツールやAPIとの連携が可能です。これらの呼び出しは並列でも順番にでも実行できます。
- *Code Generation* - Python、Java、TypeScript、C++のコード生成において高い性能を発揮します。


この例では、Mistral Large 2を使って、テキストドキュメントに対してRAGパターンを実行しています。質問は韓国語で書かれており、著者が大学入学前にどんな活動をしていたかを尋ねています。

Cohere Embeddings Modelを使って、テキストドキュメントと質問の埋め込みを作成します。このサンプルでは、ベクトルストアとしてfaiss Pythonパッケージを利用しています。

Mistralモデルに送るプロンプトには、質問と、質問に類似した取得済みのチャンクが含まれています。モデルはそれらをもとに自然な言語で回答を返します。


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Smallは、Mistralファミリーの中でもプレミア／エンタープライズカテゴリに属するモデルの一つです。名前の通り、このモデルはSmall Language Model（SLM）です。Mistral Smallを使うメリットは以下の通りです：
- Mistral LargeやNeMoなどのMistral LLMと比べてコスト削減が可能（価格が80%ダウン）
- 低レイテンシーで、MistralのLLMよりも応答が速い
- 柔軟性が高く、必要なリソースの制限が少ないため、さまざまな環境で展開可能

Mistral Smallは以下の用途に最適です：
- 要約、感情分析、翻訳などのテキストベースのタスク
- コストパフォーマンスの高さから、頻繁にリクエストが発生するアプリケーション
- レビューやコード提案など、低レイテンシーが求められるコード関連のタスク


## Mistral SmallとMistral Largeの比較

Mistral SmallとLargeのレイテンシーの違いを確認するには、以下のセルを実行してください。

応答時間に3〜5秒ほどの差が見られるはずです。同じプロンプトに対する応答の長さやスタイルにも注目してください。


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

このレッスンで紹介した他の2つのモデルと比べて、Mistral NeMoは唯一、Apache2ライセンスで無料で利用できるモデルです。

これは、Mistralの以前のオープンソースLLMであるMistral 7Bのアップグレード版と見なされています。

NeMoモデルのその他の特徴は以下の通りです。

- *より効率的なトークナイゼーション:* このモデルは、一般的に使われているtiktokenではなくTekkenトークナイザーを採用しています。これにより、より多くの言語やコードで高いパフォーマンスを発揮します。

- *ファインチューニング:* ベースモデルがファインチューニング可能な形で提供されています。これにより、ファインチューニングが必要なユースケースにも柔軟に対応できます。

- *ネイティブな関数呼び出し* - Mistral Largeと同様に、このモデルも関数呼び出しの学習がされています。これにより、オープンソースモデルとしては初期の段階でこの機能を持つ、ユニークな存在となっています。


## Mistral NeMo

このレッスンで紹介した他の2つのモデルと比べて、Mistral NeMoは唯一、Apache2ライセンスで無料で利用できるモデルです。

これは、Mistralの以前のオープンソースLLMであるMistral 7Bのアップグレード版と見なされています。

NeMoモデルのその他の特徴は以下の通りです。

- *より効率的なトークナイゼーション:* このモデルは、一般的に使われているtiktokenではなくTekkenトークナイザーを採用しています。これにより、より多くの言語やコードで高いパフォーマンスを発揮できます。

- *ファインチューニング:* ベースモデルがファインチューニング可能です。これにより、ファインチューニングが必要なユースケースでも柔軟に対応できます。

- *ネイティブな関数呼び出し* - Mistral Largeと同様に、このモデルは関数呼び出しの学習も行われています。これにより、オープンソースモデルとしては初期の段階でこの機能を持つ、ユニークな存在となっています。


### トークナイザーの比較

このサンプルでは、Mistral NeMoがどのようにトークナイズを行うかをMistral Largeと比較して見ていきます。

どちらのサンプルも同じプロンプトを使っていますが、NeMoの方がMistral Largeよりも少ないトークン数を返すことが分かるはずです。


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## 学びはここで終わりではありません。旅を続けましょう

このレッスンを終えたら、[Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) をチェックして、生成AIの知識をさらに深めましょう！



---

**免責事項**:  
本書類は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性には努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文（元の言語の文書）が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は一切の責任を負いかねます。
